# YOLOv8 ile Nesne Tespiti
### Gerekli Kütüphaneler

In [1]:
import rclpy                    #Python kodunun ROS2 sistemiyle konuşmasını sağlayan ana kütüphane
from rclpy.node import Node
import cv2                      #Görüntü matrislerini işlemek için OpenCV kütüphanesi
from ultralytics import YOLO    #YOLO yapay zeka modelini kullanmak için gerekli sınıf.

# Sınıf Tanımlama
```class YoloCameraNode(Node)```:
- Python dilinde extend işlemi parantezle gerçekleştirilir.
- Bu örnekte de 'YoloCameraNode' sınıfı 'Node' sınıfından topic yayınlama, log alma vs. miras almıştır.
# Yapıcı Metot
```__init__(self)```:
- Javada yapıcı metotlar sınıfla aynı isme sahip olurken burada standart bir isim olan ```__init__``` ifadesini kullanır.
- Self Nedir?
    - Javada sınıfın kendi değişkenlerini kullanırken 'this.variable' diyebiliriz ancak 'this' yazmak zorunlu değildir.
    - Ancak Python programlama dilinde 'self' zorunludur. Metodun ilk parametresi daima o anki nesnenin kendisini temsil eder. 'self' olmadan sınıf içindeki hiçbir değişkene veya metoda erişilemez.

```super().__init__('...')```:
- Javadaki 'super()' ile birebir aynıdır.
- Miras aldığımız ebeveyn sınıfın (Node) yapıcısını çağırır ve ROS sistemine "Benim node adım yolo_camera_node olsun" der.
### Değişken Tanımlama
- 'self.model' Java'da sınıfın en tepesinde private YOLO model; diye tanımlardın. Python'da buna gerek yok. self.degiskenAdi dediğin anda o değişken sınıfın bir parçası (attribute) olur ve hafızada yer tutar.

```self.model = YOLO("yolov8n.pt")```:
- Burada yapay zeka modelini yüklüyoruz.

```self.cap = cv2.VideoCapture(0, cv2.CAP_V4L2)```:
- Kamerayı başlatmaya yarayan OpenCV fonksiyonudur.
- Sıfır genellikle varsayılan laptop kamerasını ifade eder.
- 'cv2.CAP_V4L2' kamerayı hangi sürücü ile çalıştıracağını söyler.
### Zamanlayıcı (ROS'un Kalbi)
```self.timer = self.create_timer(0.033, self.timer_callback)```:
- C'de veya basit Java uygulamalarında while(true) döngüsü kurarsın. ROS'ta bunu yapmak tehlikelidir çünkü program kilitlenebilir.
- ROS'ta biz sisteme bir alarm kuruyoruz:
    - 0.033: Periyot (saniye).
    - self.timer_callback: "Süre dolduğunda çalıştırılacak fonksiyonun adı" (Callback Function).
## Callback Fonksiyonu (Ana İşlem)
```ret, frame = self.cap.read()```:
- Pythonda C ve Java'dan farklı olarak bir fonksiyon birden fazşa değer döndürebilir.
- 'read()' fonksiyonu 2 şey döndürür:
    - 'ret' (boolean): Okuma başarılı mı? (True/False)
    - 'frame' (Array): Resmin kendisi (Piksel matrisi)

```if ret: result = self.model(frame, veerbose=False)```:
- Eğer okuma başarılıysa, resmi modele veriyoruz. 'results' değişkeni, tespit edilen tüm nesnelerin listesini tutar.

```annotated_frame = result[0].plot()```:
- 'results[0]': Video tek kare olduğu için ilk sonucu alıyoruz.
- '.plot()': YOLO'nun kendi fonksiyonu. Resmin üzerine kutuları çizer ve bize "çizilmiş halini" geri verir.

```cv2.imshow("YOLOv8", annotated_frame)```:
- 'imshow()': Ekrana pencere açar.


In [2]:
class YoloCameraNode(Node):
    def __init__(self):
        super().__init__('yolo_camera_node')
        self.get_logger().info('YOLO Kamera Node Started')

        self.model = YOLO("yolov8n.pt") 

        self.cap = cv2.VideoCapture(0, cv2.CAP_V4L2)

        self.timer = self.create_timer(0.033, self.timer_callback)

    def timer_callback(self):
        ret, frame = self.cap.read()
        
        if ret:
            results = self.model(frame, verbose=False)

            annotated_frame = results[0].plot()

            cv2.imshow("YOLOv8 Nesne Tespiti", annotated_frame)
            cv2.waitKey(1)

## Main Fonksiyonu ve Giriş Noktası
```rclpy.init(args=args)```:
- ROS2 iletişim altyapısını başlatır.

```node = YoloCameraNode()```:
- 'YoloCameraNode' sınıfından bir nesne üretir.
- ```__init__``` burada çalışır.

```rclpy.spin(node)```:
- Node'u canlı tutar ve gelen sinyalleri (bizim timer'ımızı) dinler.
- 'spin' demezsek program yukarıdan aşağıya akar ve 'main' bittiği anda kapanır.

In [3]:
def main(args=None):
    rclpy.init(args=args)
    node = YoloCameraNode()
    rclpy.spin(node)

## Tek Başına Çalıştırılabilir Hale Getirme
- Python dosyaları, hem direkt çalıştırılabilir hem de başka bir dosyaya kütüphane gibi import edilebilir.
- Bu satır der ki: "Eğer bu dosya doğrudan çalıştırılıyorsa o zaman 'main()' fonksiyonunu çalıştır."
- "Eğer başka bir kod bu dosyayı import ediyorsa, main()'i çalıştırma, sadece içindeki sınıfları kullanıma hazırla."

In [ ]:
if __name__ == '__main__':
    main()

[INFO] [1766946306.706294512] [yolo_camera_node]: YOLO Kamera Node Started
